In [94]:
from collections import defaultdict, Counter
from intervaltree import Interval, IntervalTree
import itertools
import functools
import math
import numpy as np
import re
from typing import Callable, TypeVar

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

In [99]:
data1 = data(1)

def day1(match):
    sum = 0
    for line in data1:
        matched = match(line)
        first, last = matched[0], matched[-1]
        # print(first, last, sum)
        sum += int(first + last)
    return sum

print(day1(lambda x: re.findall(r'\d', x)))

def part2match(x):
    change = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9',
    }
    matched = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', x)
    return [change[match] if match in change else match for match in matched]

day1(part2match)
    

55816


54980

In [148]:
def format2(line):
    def tuplefy(round):
        result = [0,0,0]
        for i in round:
            result[0 if 'blue' in i else 1 if 'green' in i else 2] += int(i.split(' ')[0])
        return result

    line = line.split(': ')[-1]
    rounds = line.split('; ' )
    return np.array([tuplefy(x.split(', ')) for x in rounds])

data2 = data(2, format2)

def day2_1(start):
    result = 0
    for i, game in enumerate(data2):
        possible = np.subtract(start, game)
        if np.any(possible < 0):
            continue
        result += i+1
    return result

print(day2_1((14, 13, 12)))

def day2_2():
    result = 0
    for game in data2:
        requirements = np.amax(game, axis=0)
        result += functools.reduce(lambda x, y: x*y, requirements, 1)
    return result
day2_2()

2377


71220

In [130]:
data3 = data(3)

def day3():
    numbers = []
    for i, line in enumerate(data3):
        numbers += [(m.group(), (i, m.start())) for m in re.finditer(r'\d+', line)]

    array3 = np.pad(np.array([list(line) for line in data3]), ((1, 1), (1, 1)), constant_values='.')
    gears = defaultdict(lambda: [])

    part1 = 0
    for n, (i, j) in numbers:
        sliced = array3[i:i+3, j:j+2+len(n)]
        stringified =  ''.join(''.join(x) for x in sliced)
        if re.search(r'[^\d.]', stringified):
            part1 += int(n)
        if stars := np.where(sliced == '*'):
            for x, _ in enumerate(stars[0]):
                gears[(i+stars[0][x], j+stars[1][x])].append(int(n))

    part2 = 0
    for g in gears:
        if len(gears[g]) == 2:
            part2 += gears[g][0] * gears[g][1]

    return part1, part2

day3()

(521515, 69527306)

In [5]:
def parse4(line):
    winning, have = map(
        lambda numbers: {int(x) for x in numbers.split()},
        line.split(':')[-1].split('|')
    )
    return winning, have

data4 = data(4, parse4)

def day4(data):
    result, cards = 0, Counter()
    for i, line in enumerate(data):
        matches = line[0].intersection(line[1])
        if matches:
            result += 2**(len(matches)-1)
        for n in range(len(matches)):
            cards[i+1+n] += cards[i]+1
    return result, cards.total() + len(data)

day4(data4)


(25651, 19499881)

In [172]:
a =  IntervalTree([Interval(0, 1000)])
a.chop(1, 10)
a

IntervalTree([Interval(0, 1), Interval(10, 1000)])

In [224]:
def create_map(lines):
    name = lines[0].split(' ')[0]
    ranges = IntervalTree([Interval(0, math.inf)])
    for i, line in enumerate(lines[1:]):
        if not line:
            break
        dest, source, size = map(int, line.split())
        ranges.chop(source, source+size)
        ranges[source:source+size] = dest
    return name, ranges, lines[i+2:]

def day5_maps(data):
    seeds, data = map(int, data[0].split(': ')[-1].split()), data[2:]
 
    maps = []
    while len(data):
        name, intervals, data = create_map(data)
        maps.append(intervals)
    print(maps)

    return list(seeds), maps

seeds, data5 = day5_maps(data(5))


[IntervalTree([Interval(0, 257757572, 77244511), Interval(257757572, 453310112, 391285622), Interval(453310112, 512210869, 18343754), Interval(512210869, 568494408, 335002083), Interval(568494408, 586838162, 0), Interval(586838162, 605476380), Interval(605476380, 762191092, 2070957389), Interval(762191092, 897735089, 3100147259), Interval(897735089, 919745076, 3242241597), Interval(919745076, 1015585345, 2227672101), Interval(1015585345, 1297982331, 2323512370), Interval(1297982331, 1392195671, 2605909356), Interval(1392195671, 2139601898, 3547561069), Interval(2139601898, 2186640160, 2022435244), Interval(2186640160, 2348990120, 826809686), Interval(2348990120, 2355540461, 3235691256), Interval(2355540461, 2633818373, 2821869347), Interval(2633818373, 3166958320, 1112103608), Interval(3166958320, 3544150009, 1645243555), Interval(3544150009, 3732587455, 605476380), Interval(3732587455, 3734071338, 2069473506), Interval(3734071338, 4017380823, 3264251584), Interval(4017380823, 40502766

In [179]:
lowest = math.inf
for seed in seeds:
    # print(seed)
    for interval in data5:
        start, end, dest = next(iter(interval.at(seed)))
        # print(start, end, dest, seed)
        seed = seed if dest is None else (seed-start)+dest
    lowest = min(lowest, seed)
print(lowest)

35


In [223]:
lowest = math.inf
for i in range(len(seeds)//2):
    seedrange = IntervalTree([Interval(seeds[i*2], seeds[i*2]+seeds[i*2+1])])
    print(seedrange)

    for interval in data5:
        for seed in seedrange:
            seedstart, seedend = seed[0], seed[1]
            intersections = interval[seedstart:seedend]
            # print(intersections, seed)
            newrange = IntervalTree()
            for hit in intersections:
                start, end, dest = hit
                if dest is not None:
                    newstart = max(seedstart,start)-start+dest
                    newend = min(seedend, end)-start+dest
                # print('hit', start, end, seedstart, seedend, newstart,newend)
                newrange.addi(newstart, newend)
            # print('a', seedrange, 'new', newrange)
        newrange.merge_overlaps()
        seedrange = newrange
    print(seedrange)
    for loc in seedrange:
        print(loc[0], lowest)
        lowest = min(loc[0], lowest)
print(lowest)

IntervalTree([Interval(79, 93)])
IntervalTree([Interval(46, 57), Interval(60, 61)])
60 inf
46 60
IntervalTree([Interval(55, 68)])
IntervalTree([Interval(86, 90)])
86 46
46
